In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import argparse
import importlib
import data

from glob import glob
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

importlib.reload(data)

In [ ]:
#load img
#to load whole dataset will need to write a torch Dataset class

import data; importlib.reload(data)
batch_size=8

datamodule=data.IRAVENDataModule(batch_size=batch_size)
datamodule.setup()
train_dataloader,val_dataloader,test_dataloader=datamodule.train_dataloader(),datamodule.val_dataloader(),datamodule.test_dataloader()

print(train_dataloader.dataset.__len__(),test_dataloader.dataset.__len__(),val_dataloader.dataset.__len__())

datamodule=data.IRAVENDataModule(batch_size=batch_size,split=(90,10,0))
datamodule.setup()
train_dataloader,val_dataloader,test_dataloader=datamodule.train_dataloader(),datamodule.val_dataloader(),datamodule.test_dataloader()

print(train_dataloader.dataset.__len__(),test_dataloader.dataset.__len__(),val_dataloader.dataset.__len__())


In [ ]:
import collections

train_dataset_fnames=train_dataloader.dataset.filenames
test_dataset_fnames=test_dataloader.dataset.filenames


def get_type_strs(structure):
    types=structure[3]
    types_str=eval(str(types)[1:])
    return types_str

types_str_dataset=[]
for file in test_dataset_fnames:
    structure=np.load(file)['structure']
    types_str_dataset.append(get_type_strs(structure))
    






    frequency=collections.Counter(types_str_dataset)
